# Základy Entity Frameworku

**autor: Erik Král ekral@utb.cz**

---



Entity Framework (EF) slouží k objektově relačnímu mapování. Což znamená, že můžeme pracovat s objekty v paměti a EF nám vygeneruje příkazy pro databází. Díky tomu také nejsme závislí na konkrétním typu databáze. S pomocí technologie **LINQ to Entities** (entita je třída reprezentující řádek tabulky v databázi) potom pracujeme s databází obdobným způsobem jako s objekty pomocí **LINQ to objects**.

Návod pro Entity Framework: [Getting Started with EF Core](https://learn.microsoft.com/en-us/ef/core/get-started/overview/first-app?tabs=netcore-cli). 

## Příklad

V následujícím příkladu definujteme třídu Student a pomocí migrací vytvoříme Sqlite databázi.

### 1. Definice třídy Student

Nejprve si nadefinujeme třídu `Student`. Ve třídě nechceme mít parametrický konstruktor, což by nám mohlo komplikovat práci s Entity Frameworkem. Property představují sloupce tabulky a ```Id``` je dle jmenných konvencí primární klíč. Jako alternativu pro parametrický konstruktor v příkladu použiváme klíčové slovo ```required```, které říká, že ```Jmeno``` a ```Prijmeni``` musi mit přiřazenou hodnotu nejpozději v Object Initializeru.

In [1]:
public class Student
{
    public int Id { get; set; } // Primární klíč dle jmenných konvencí
    public required string Jmeno { get; set; }     
    public required string Prijmeni { get; set; }     
}

Student student = new()
{
    Jmeno = "Andrea",
    Prijmeni = "Nova"
};

Pokud chceme používat konkrétní databázi s Entity Frameworkem, tak musím do projektu přidat providera pro tuto databázi. Provider je většinou knihovna distriovaná jako nuget balíček. Následující příkaz nainstaluje nuget balíček, konrétně EF database provider pro databázi Sqlite.

Příkaz musíte spustit v adresáři, kde se nachází projekt, tedy soubor s příponou *.csproj.

```powershell
dotnet add package Microsoft.EntityFrameworkCore.Sqlite
```

Dále definujeme potomka třídy ```DbContext``` a kolekce ```DbSet``` potom definuje tabulku v databázi.

In [2]:
#r "nuget:Microsoft.EntityFrameworkCore.Sqlite, 8.*"

using Microsoft.EntityFrameworkCore;

public class StudentContext : DbContext
{
    public DbSet<Student> Students { get; set; }
}

Installed Packages Microsoft.EntityFrameworkCore.Sqlite, 8.0.11

Pomocí přetížené metody OnConfiguring potom nakonfigurujeme databázi, konrétně zadáme connection string. 

Jednoduchý zápis:

In [4]:
public class StudentContext : DbContext
{
    public DbSet<Student> Students { get; set; }

    protected override void OnConfiguring(DbContextOptionsBuilder optionsBuilder)
    {
        optionsBuilder.UseSqlite("Data Source=studenti.db");
    }
}

Zápis, který ukládá databázi do dokumentů uživatele a používá connection string builder aby nedošlo k chybnému zápisu.

In [7]:
using System.IO;
using Microsoft.Data.Sqlite;

public class StudentContext : DbContext
{
    public DbSet<Student> Students { get; set; }

    protected override void OnConfiguring(DbContextOptionsBuilder optionsBuilder)
    {
        var folder = Environment.SpecialFolder.MyDocuments;
        string folderPath = Environment.GetFolderPath(folder);
        string filePath = Path.Join(folderPath, "studenti3.db");

        SqliteConnectionStringBuilder csb = new SqliteConnectionStringBuilder
        {
            DataSource = filePath
        };

        optionsBuilder.UseSqlite(csb.ConnectionString);
    }
}

Alternativně můžeme předat `DbContextOptions<ApplicationDbContext>` v konstruktoru. Díky tomu, můžeme mimo třídu zvolit i jiného providera a tedy používat jinou databázi. To je výhodné například při testování.

In [3]:
public class StudentContext : DbContext
{
    public DbSet<Student> Students { get; set; }

    public StudentContext(DbContextOptions<StudentContext> options) : base(options)
    {

    }
}

DbContextOptions<StudentContext> options = new DbContextOptionsBuilder<StudentContext>().UseSqlite("Data Source=studenti.db").Options; 

/* using(StudentContext context = new StudentContext(options))
{

}*/